#!pip install geopandas shapely

pip install pyarrow
pip install fastparquet

!pip install simplejson

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import geopandas as gpd
import pyarrow.parquet as pq
import json

Extracting raw data :
 - Data source 1 : Transactional data for NYC trip with 1 row for i individual trip. The dataa is in Apache parquet format
 - Data source 2 : csv data with Taxi zone lookup information 
 - Data source 2 : GIS Spatial data containg shaplefile (.shp) for geomatic coordinates in a zipfolder

## Data Loading : Extraction 

In [27]:
# Read Parquet Trip data into Pandas DataFrame

df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

# additinal trip data

taxizonelookup = pd.read_csv('https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv')

#  Geospatial data from zip folder to get coordinates
zipfile = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip'
location_dim = gpd.read_file(zipfile)  

####  Exploring raw data

In [28]:
print(df.head())
print(df.shape)

print(df.columns)
print (df.nunique())

print(df.info())
print(df.describe())

# Categorical columns
print(df['PULocationID'].nunique())
print(df['DOLocationID'].nunique())
print(df['passenger_count'].nunique())
print(df['trip_distance'].nunique())
'''

# trip_type column exists inonly few dataset
print(df['trip_type'].nunique())
print(df['trip_type'].unique())
print(df['trip_type'].value_counts()) ## <3% rows of type 2
'''

print(df['airport_fee'].unique())
print(df['congestion_surcharge'].unique())         
print(df['improvement_surcharge'].unique())   

print(df['tolls_amount'].nunique())            
print(df['tip_amount'].nunique())            
print(df['mta_tax'].nunique())        
print(df['extra'].nunique())   
print(df['fare_amount'].nunique())     

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   
3           1.90         1.0                  N           138             7   
4           1.43         1.0                  N           107            79   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


## Data Transformation : Dimension Modeling (Star schema)

Cleaning , Removing Duplicates, Correcting Datatype, reset index , Create Primary and Surrogate Key

In [32]:
# Keeping granularity only till hour 
df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].dt.floor('H')
df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].dt.floor('H')

### - converting ID columns to INT datatype (from float)  + dealing with nan

In [33]:
### Passenger count column 
print(df['passenger_count'].value_counts())

df[df['passenger_count'].isna()]  # view the rows with nan

print(df['passenger_count'].isna().sum()) #total rows with nana 
 
# since passanger_count is a dim table 
# assign 10 (10 surrogate id) to nan vale 

df['passenger_count'] = df['passenger_count'].fillna(10).round().astype(int)

1.0    2261400
2.0     451536
3.0     106353
4.0      53745
0.0      51164
5.0      42681
6.0      28124
8.0         13
7.0          6
9.0          1
Name: passenger_count, dtype: int64
71743


In [34]:
### RatecodeID column 

print(df['RatecodeID'].value_counts())
print(df['RatecodeID'].unique())
print(df['RatecodeID'].isna().sum()) # total nan values in ratecodeID column
print(df[df['RatecodeID'].isna()].head(5)) # print rows with nan 
# assign 10 (10 surrogate id) to nan vale 

df['RatecodeID'] = df['RatecodeID'].fillna(10).round().astype(int)

1.0     2839305
2.0      114239
5.0       15043
99.0      13106
3.0        8958
4.0        4366
6.0           6
Name: RatecodeID, dtype: int64
[ 1.  2.  4. 99.  5.  3.  6. nan]
71743
         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
2995023         1           2023-01-01   2023-01-01 00:00:00               10   
2995024         1           2023-01-01   2023-01-01 01:00:00               10   
2995025         2           2023-01-01   2023-01-01 01:00:00               10   
2995026         2           2023-01-01   2023-01-01 00:00:00               10   
2995027         2           2023-01-01   2023-01-01 00:00:00               10   

         trip_distance  RatecodeID store_and_fwd_flag  PULocationID  \
2995023           0.00         NaN               None           142   
2995024           0.00         NaN               None           237   
2995025           0.86         NaN               None            24   
2995026           5.36         NaN            

### checking numeric columns with negetive amount


In [35]:
# checking numeric columns with negetive amount
df[df['total_amount']<0]['payment_type'].unique()  
df[df['total_amount']<0]['VendorID'].unique()  # negetive value is from vendor 2
df[df['total_amount']<0]['store_and_fwd_flag'].unique()  

array(['N', 'Y', None], dtype=object)

In [36]:
# creating a function to convert only negetive values in dataset

def convert_negetive_only(x):
    if (pd.notna(x) and x < 0):
        return -x
    else :
        return x

In [37]:
df['improvement_surcharge']  = df['improvement_surcharge'].apply(convert_negetive_only)
df['congestion_surcharge'] = df['congestion_surcharge'].apply(convert_negetive_only)
df['airport_fee'] = df['airport_fee'].apply(convert_negetive_only)
df['tolls_amount'] = df['tolls_amount'].apply(convert_negetive_only)           
df['tip_amount'] = df['tip_amount'].apply(convert_negetive_only)          
df['mta_tax'] = df['mta_tax'].apply(convert_negetive_only)         
df['extra'] = df['extra'].apply(convert_negetive_only)   
df['fare_amount'] = df['fare_amount'].apply(convert_negetive_only)    
df['total_amount'] = df['total_amount'].apply(convert_negetive_only)  

### drop duplicates and reset index , set primary key

In [38]:
df = df.drop_duplicates().reset_index(drop=True)  # drop duplicates for whole dataset
df['trip_id'] = df.index #set new index value and save it as new PK column

In [39]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_id
0,2,2023-01-01,2023-01-01 00:00:00,1,0.97,1,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0
1,2,2023-01-01,2023-01-01 01:00:00,1,1.10,1,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,1
2,2,2023-01-01,2023-01-01 00:00:00,1,2.51,1,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,2
3,1,2023-01-01,2023-01-01 00:00:00,0,1.90,1,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,3
4,2,2023-01-01,2023-01-01 00:00:00,1,1.43,1,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,4


## DIMENSION MODELING

### Creating DIM_date

In [40]:
#Generate a Range of Dates: dates we want to include in the dimension table. This range should cover the time span of the data.

# Granularity to hour
start_date = df['tpep_pickup_datetime'].min().floor('H')
print(f" min dattime in raw data :{start_date}")
end_time = df['tpep_dropoff_datetime'].max().floor('H')
print(f" max date in raw data :{end_time}")

##create a series of required date range   
date_range = pd.date_range( start=start_date, end=end_time, freq='H' )  # D = day H= hour T= minutes 


 min dattime in raw data :2008-12-31 23:00:00
 max date in raw data :2023-02-02 09:00:00


In [41]:
date_dim = pd.DataFrame({'date':date_range})
date_dim = date_dim.reset_index(drop=True)

# using surrogatee= key as a pk in dim
date_dim['date_id'] = date_dim.index 
#date_dim.head()


date_dim['year'] = date_dim['date'].dt.year
date_dim['month'] = date_dim['date'].dt.month
date_dim['day'] = date_dim['date'].dt.day
date_dim['weekday'] = date_dim['date'].dt.weekday  # Monday is 0 and Sunday is 6
date_dim['day_name'] = date_dim['date'].dt.day_name()
date_dim['hour'] = date_dim['date'].dt.hour

date_dim = date_dim[['date_id','date', 'year', 'month', 'day', 'weekday', 'day_name', 'hour']]
date_dim.head()

,date_id,date,year,month,day,weekday,day_name,hour
0,0,2008-12-31 23:00:00,2008,12,31,2,Wednesday,23
1,1,2009-01-01 00:00:00,2009,1,1,3,Thursday,0
2,2,2009-01-01 01:00:00,2009,1,1,3,Thursday,1
3,3,2009-01-01 02:00:00,2009,1,1,3,Thursday,2
4,4,2009-01-01 03:00:00,2009,1,1,3,Thursday,3


### Creating Vendor_dim

In [42]:
vendordict = {1: 'Creative Mobile Technologies', 2: 'VeriFone Inc'}

vendor_dim = pd.DataFrame({ 'VendorID':list(vendordict.keys()) , 'Vendor_name':list(vendordict.values()) })

# primary key of is a surrogate key ; considering this a SCD-2
vendor_dim['vendor_PK'] = range(1, len(vendor_dim) + 1)

vendor_dim = vendor_dim[['vendor_PK', 'VendorID', 'Vendor_name']]

vendor_dim.head()


,vendor_PK,VendorID,Vendor_name
0,1,1,Creative Mobile Technologies
1,2,2,VeriFone Inc


### Creating RATE_DIM

In [15]:
rate_dict = {
    1: 'Standard rate',
    2:'JFK',
    3:'Newark',
    4: 'Nassau or Westchester',
    5: 'Negotiated fare',
    6:'Group ride',
10:'nan',
99:'no information'} 

Ratecode_dim  =  pd.DataFrame({'RatecodeID':list(rate_dict.keys())}).reset_index(drop=True)
Ratecode_dim['RatecodeType']  =     Ratecode_dim['RatecodeID'].map(rate_dict) # natural key 
Ratecode_dim['Ratedim_id'] = Ratecode_dim.index #surrogate pk

Ratecode_dim = Ratecode_dim[['Ratedim_id', 'RatecodeID', 'RatecodeType']]

Ratecode_dim

,Ratedim_id,RatecodeID,RatecodeType
0,0,1,Standard rate
1,1,2,JFK
2,2,3,Newark
3,3,4,Nassau or Westchester
4,4,5,Negotiated fare
5,5,6,Group ride
6,6,10,nan
7,7,99,no information


### Creating PAYMENT_DIM

In [16]:
payment_dict = {
        0: 'no information',
        1: 'Credit card',
        2: 'Cash',
        3: 'No charge',
        4: 'Dispute',
        5: 'Unknown',
        6: 'Voided trip'}

payment_dim = pd.DataFrame({'Payment_typeID':list(payment_dict.keys()),
                           'Payment_type':list(payment_dict.values())})
payment_dim

,Payment_typeID,Payment_type
0,0,no information
1,1,Credit card
2,2,Cash
3,3,No charge
4,4,Dispute
5,5,Unknown
6,6,Voided trip


### Creating LOCATION_DIM

In [23]:

#contains service_zone column
#taxizonelookup = pd.read_csv('https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv')
taxizonelookup.head(10)

#converting data into dictionary 
tzlookup_dict = taxizonelookup.set_index('LocationID')['service_zone'].to_dict()
tzlookup_dict

{1: 'EWR',
 2: 'Boro Zone',
 3: 'Boro Zone',
 4: 'Yellow Zone',
 5: 'Boro Zone',
 6: 'Boro Zone',
 7: 'Boro Zone',
 8: 'Boro Zone',
 9: 'Boro Zone',
 10: 'Boro Zone',
 11: 'Boro Zone',
 12: 'Yellow Zone',
 13: 'Yellow Zone',
 14: 'Boro Zone',
 15: 'Boro Zone',
 16: 'Boro Zone',
 17: 'Boro Zone',
 18: 'Boro Zone',
 19: 'Boro Zone',
 20: 'Boro Zone',
 21: 'Boro Zone',
 22: 'Boro Zone',
 23: 'Boro Zone',
 24: 'Yellow Zone',
 25: 'Boro Zone',
 26: 'Boro Zone',
 27: 'Boro Zone',
 28: 'Boro Zone',
 29: 'Boro Zone',
 30: 'Boro Zone',
 31: 'Boro Zone',
 32: 'Boro Zone',
 33: 'Boro Zone',
 34: 'Boro Zone',
 35: 'Boro Zone',
 36: 'Boro Zone',
 37: 'Boro Zone',
 38: 'Boro Zone',
 39: 'Boro Zone',
 40: 'Boro Zone',
 41: 'Boro Zone',
 42: 'Boro Zone',
 43: 'Yellow Zone',
 44: 'Boro Zone',
 45: 'Yellow Zone',
 46: 'Boro Zone',
 47: 'Boro Zone',
 48: 'Yellow Zone',
 49: 'Boro Zone',
 50: 'Yellow Zone',
 51: 'Boro Zone',
 52: 'Boro Zone',
 53: 'Boro Zone',
 54: 'Boro Zone',
 55: 'Boro Zone',
 56: 'Bor

In [24]:
#zipfile = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip'
#location_dim = gpd.read_file(zipfile)  

# Extract Latitude and Longitude
location_dim['latitude'] = location_dim.geometry.centroid.y
location_dim['longitude'] = location_dim.geometry.centroid.x

# appending service_zone column in location dim
location_dim['service_zone'] = location_dim['LocationID'].map(tzlookup_dict)

#reordering columns
location_dim = location_dim[['LocationID','OBJECTID','zone','borough','service_zone','latitude','longitude','geometry','Shape_Leng','Shape_Area']]


In [33]:
location_dim.head()

,LocationID,OBJECTID,zone,borough,service_zone,latitude,longitude,geometry,Shape_Leng,Shape_Area
0,1,1,Newark Airport,EWR,EWR,191376.749531,9.359968e+05,"POLYGON ((933100.918 192536.086, 933091.011 19...",0.116357,0.000782
1,2,2,Jamaica Bay,Queens,Boro Zone,164018.754403,1.031086e+06,"MULTIPOLYGON (((1033269.244 172126.008, 103343...",0.433470,0.004866
2,3,3,Allerton/Pelham Gardens,Bronx,Boro Zone,254265.478659,1.026453e+06,"POLYGON ((1026308.770 256767.698, 1026495.593 ...",0.084341,0.000314
3,4,4,Alphabet City,Manhattan,Yellow Zone,202959.782391,9.906340e+05,"POLYGON ((992073.467 203714.076, 992068.667 20...",0.043567,0.000112
4,5,5,Arden Heights,Staten Island,Boro Zone,140681.351376,9.318714e+05,"POLYGON ((935843.310 144283.336, 936046.565 14...",0.092146,0.000498


### Creating FACT table

In [23]:
df.columns
df.shape
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_id
0,2,2023-01-01,2023-01-01 00:00:00,1,0.97,1,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0
1,2,2023-01-01,2023-01-01 01:00:00,1,1.10,1,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,1
2,2,2023-01-01,2023-01-01 00:00:00,1,2.51,1,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,2
3,1,2023-01-01,2023-01-01 00:00:00,0,1.90,1,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,3
4,2,2023-01-01,2023-01-01 00:00:00,1,1.43,1,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,4


In [27]:
fact_table = df.merge( date_dim, left_on='tpep_pickup_datetime', right_on='date', how='left') \
                .merge( date_dim, left_on='tpep_dropoff_datetime', right_on='date', how='left') \
                .merge( vendor_dim, left_on='VendorID', right_on='VendorID', how='left') \
                .merge( Ratecode_dim, left_on='RatecodeID', right_on='RatecodeID', how='left') \
                .merge( payment_dim, left_on='payment_type', right_on='Payment_typeID', how='left') \
                    [['trip_id', 'date_id_x',  'date_id_y',  'DOLocationID','PULocationID',  'vendor_PK', 'Ratedim_id', 'Payment_typeID'
                           ,'passenger_count', 'trip_distance', 'store_and_fwd_flag','fare_amount', 'extra',
                           'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
                            'congestion_surcharge', 'airport_fee', 'total_amount']]




In [30]:
fact_table.rename(columns={'date_id_x': 'tpep_pickup_datetime'}, inplace=True)
fact_table.rename(columns={'date_id_y': 'tpep_dropoff_datetime'}, inplace=True)


In [32]:
print(fact_table.shape)
print(fact_table.columns)
fact_table.head() 

(3041801, 20)
Index(['trip_id', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'DOLocationID', 'PULocationID', 'vendor_PK', 'Ratedim_id',
       'Payment_typeID', 'passenger_count', 'trip_distance',
       'store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'congestion_surcharge',
       'airport_fee', 'total_amount'],
      dtype='object')


,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,DOLocationID,PULocationID,vendor_PK,Ratedim_id,Payment_typeID,passenger_count,trip_distance,store_and_fwd_flag,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,airport_fee,total_amount
0,0,122713,122713,141,161,2,0,2,1,0.97,N,9.3,1.00,0.5,0.00,0.0,1.0,2.5,0.00,14.30
1,1,122713,122714,237,43,2,0,1,1,1.10,N,7.9,1.00,0.5,4.00,0.0,1.0,2.5,0.00,16.90
2,2,122713,122713,238,48,2,0,1,1,2.51,N,14.9,1.00,0.5,15.00,0.0,1.0,2.5,0.00,34.90
3,3,122713,122713,7,138,1,0,1,0,1.90,N,12.1,7.25,0.5,0.00,0.0,1.0,0.0,1.25,20.85
4,4,122713,122713,79,107,2,0,1,1,1.43,N,11.4,1.00,0.5,3.28,0.0,1.0,2.5,0.00,19.68


In [40]:
dic = {'a':123, 'b': 900, 'c': 456 }
print(dic.items())


for x,y in dic.items() :
    print(f" key = {x} , value = {y} ")
    z = 'this is the youtube Dynamic assigmen for x = {}'.format(y)
    print(z)
     

dict_items([('a', 123), ('b', 900), ('c', 456)])
 key = a , value = 123 
this is the youtube Dynamic assigmen for x = 123
 key = b , value = 900 
this is the youtube Dynamic assigmen for x = 900
 key = c , value = 456 
this is the youtube Dynamic assigmen for x = 456
